# CMPE 256 Programming Assignment 

## Id : 013774228 , Sarath Chandra Makkena

### supporting libraries used : Surprise, Pandas, Numpy

In [1]:
# first step :Import libraries and load respective dataset into pandas dataframe 
# Then used surprise library fucntion load_from_df() to load from pandas dataframe
# After which building full trainset using entire dataset

import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise import Dataset, evaluate
from surprise import SVD
from surprise import SVDpp
from surprise import KNNBasic

from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV
from surprise import accuracy

df = pd.read_csv('train_newdat.csv')
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df[['user_id', 'book_id', 'rating']], reader)
trainingSet = data.build_full_trainset()


In [2]:
#Print the surprise dataset
print(data)

### In the given data set ratings are scrambled, so trying to sort in descending order

In [3]:
# sorting values in descending order and displaying first few rows using head()
df.sort_values(['rating'],ascending=False).head()

,user_id,book_id,rating
0,12726,7784,5
359885,30537,759611,5
151869,33101,375901,5
151871,13492,197084,5
648308,490,37190,5


### second step : loading test into data frame

In [4]:
#test dataset is loaded using pandas read_csv
dt = pd.read_csv('test_new.csv')

### it's important to have basic overview of tarining set we have, Based on which we can plan for pre-processing data. Now describing the given data frame to know whether any null values or NAN values present in training set.

In [5]:
df.describe()

,user_id,book_id,rating
count,700000.000000,7.000000e+05,700000.000000
mean,18895.764374,4.970340e+06,2.644934
std,10745.949583,8.443227e+06,2.051200
min,0.000000,5.000000e+00,0.000000
25%,9558.000000,7.453200e+04,0.000000
50%,18986.000000,4.349280e+05,3.000000
75%,27946.000000,6.578293e+06,4.000000
max,37230.000000,3.645941e+07,5.000000


In [6]:
# checking whether test set loaded into data frame dt.
dt.head()

,user_id,book_id
0,20989,1832332
1,37040,191139
2,36167,28449164
3,9398,24693869
4,29848,8127


In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

train = pd.read_csv('train_newdat.csv')
target = 'Disbursed'
IDcol = 'ID'


In [ ]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print "\nModel Report"
    print "Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions)
    print "AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob)
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
#Choose all predictors except target & IDcols
predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, train, predictors)

In [ ]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test2b = {
 'min_child_weight':[6,8,10,12]
}
gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=4,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2b, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2b.fit(train[predictors],train[target])

In [ ]:
modelfit(gsearch3.best_estimator_, train, predictors)
gsearch2b.grid_scores_, gsearch2b.best_params_, gsearch2b.best_score_

In [ ]:
param_test2 = {
 'max_depth':[4,5,6],
 'min_child_weight':[4,5,6]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(train[predictors],train[target])
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

### Here am trying to use GridSeachCV evaluating method availble in surprise library to figure out optimal parameter for given data set. Which takes type of prediction algorithm i.e. alogorithm class, parameter grid, error measure type, and data set split proportion. As am going to use SVD (Singular Value Decomposition) predication measuring algorithm, which's having many tuning params, so parameters range for  lr_all (learning rate), reg_all (regularisation),n_factors into GridsearchCV. it works like bruteforce technique, check as possible combinations with-in the range and returns optimal parameters givin better RMSE.

In [8]:
# listing out parameters into param_grid, inorder to pass it as arguements for GridSearchCV parameters
# along with type of predction algorithm , parameters are fed for gridsearch to find optimal tuning parameters for given dataset.
# resultant is to print the best fit parameters 
param_grid = {'lr_all' : [.001, .1], 'reg_all' : [.001, .5], 'n_factors' : [150, 400]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv = 5)
gs.fit(data)
print(gs.best_params['rmse'])

{'lr_all': 0.1, 'reg_all': 0.5, 'n_factors': 400}


### As optimal parameters available for SVD, Time to pulgin those parameters and run the algorithm to train our model. Once training is finished accuracy is measured using evaluation method RMSE. lower the rmse value better the prediction.

In [9]:
#SVD() is used as algorithm  with learning rate=0.1, regulation rate=0.5 and factors to be 400
# print RMSE for each stage as am splitting dataset into 5 parts using KFold.

algo1=SVD(n_epochs=50,lr_all=0.1,reg_all =0.5,n_factors =400)

kf = KFold(n_splits=5)

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo1.fit(trainingSet)
    predictions = algo1.test(testset)
    accuracy.rmse(predictions, verbose=True)

RMSE: 1.4405
RMSE: 1.4412
RMSE: 1.4388
RMSE: 1.4367
RMSE: 1.4370


### For final submission into Kaggle criteria for uploading result, criteria is to concatenate two columns (user_id,book_id) into one coulmn, Here am using numpy to append values. 

In [10]:
# creating list result to store predicted values 
# ID list contains the concatenated ID-User and book values
import numpy as np
result=[]
result1=[]
id=[]
for index, row in dt.iterrows():
    id.append(str((row['user_id']))+'-'+str((row['book_id'])))
    result1.append(algo1.predict(row['user_id'], row['book_id']).est)


In [11]:
#Create a new dataframe to store the ID and Predicted ratings
result=pd.DataFrame({'user_id-book_id':pd.Series(id),'rating':pd.Series(result1) })

In [12]:
print(result)

       user_id-book_id    rating
0        20989-1832332  0.396672
1         37040-191139  2.588991
2       36167-28449164  1.301240
3        9398-24693869  0.908403
4           29848-8127  2.464246
5          14822-38674  4.201941
6        11384-1487686  3.807458
7       15022-11630978  1.800384
8           5692-19543  2.772055
9           3522-34268  2.109552
10        19940-824204  4.360288
11        13048-233818  1.368019
12          6334-17496  1.265943
13        12090-955942  1.184463
14        18985-902750  1.512923
15         32410-20694  3.472420
16        9476-1812559  3.733182
17       29226-4634866  1.869722
18        13106-105549  4.316514
19      14303-32078585  0.169351
20          20799-8127  3.617313
21        35002-850253  2.643288
22          3150-28195  2.724413
23        6640-9762941  2.815556
24        36273-511689  2.926007
25      23290-30201521  4.320704
26        17518-290499  2.187012
27       20755-8521055  1.164048
28       35014-4812114  2.508664
29        

In [13]:
# Export result into CSV file
result.to_csv('Results_Sar_rep.csv',index=False)

### References
- https://surprise.readthedocs.io/en/stable/index.html
- https://surprise.readthedocs.io/en/stable/prediction_algorithms.html
- https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD 
- https://surprise.readthedocs.io/en/stable/evaluate.html?highlight=GridSearchCV#surprise.evaluate.evaluate